In [ ]:
from tqdm import tqdm
import numpy as np
from numpy.linalg import matrix_power
import networkx as nx
import scipy as sp
from copy import deepcopy

## Part 1 +2

Part 1 works, for part 2 I would have to improve the speed of the compact_tower() function and run it many times, each time removing one block, and then check the differences.
As currently compact_tower() takes about 2 minutes, this is not feasible

In [ ]:
dir_dict= {0 : 'x',
           1 : 'y',
           2 : 'z',
          -1 : 'cube'
           }

class Block:
    def __init__(self, first_edge, second_edge):
        first_edge = [int(x) for x in first_edge.split(',')]
        second_edge = [int(x) for x in second_edge.split(',')]
        self.edge1, self.edge2, self.dir = self.__order_edge_smaller_greater_plus_direction(first_edge, second_edge)
        #self.dir  0 =x, 1 = y, 2 = z, -1 = cube block
        self.length = self.edge2[self.dir] - self.edge1[self.dir] +1
        self.n_blocks_above_falling = 0
        
    def __order_edge_smaller_greater_plus_direction(self, edge1, edge2):
        for idx in range(3):
            if edge1[idx] >edge2[idx]:
                return edge2, edge1, idx
            if edge1[idx] < edge2[idx]:
                return edge1, edge2, idx
        # if it's a 1 cube block   
        return edge1, edge2, -1
    
    def unravel_block(self):
        is_x = self.dir == 0
        is_y = self.dir == 1
        is_z = self.dir == 2
        return [[self.edge1[0]+idx*is_x, self.edge1[1]+idx*is_y, self.edge1[2]+idx*is_z] for idx in range(self.length) ]
    
    def __lt__(self, block2):       
        for idx in range(2, -1, -1):
            if self.edge1[idx]> block2.edge1[idx]:
                return False
            elif self.edge1[idx]< block2.edge1[idx]:
                return True
        return True
    
    def __gt__(self, block2):       
        for idx in range(2, -1, -1):
            if self.edge1[idx]< block2.edge1[idx]:
                return False
            elif self.edge1[idx]> block2.edge1[idx]:
                return True
        return True
    
    def gen_lower_block(self):
        new_block = deepcopy(self)
        new_block.edge1[2] -= 1
        new_block.edge2[2] -= 1
        return new_block
    
    
    def does_intersect(self, block2):
        cubes_self = self.unravel_block()
        cubes_block2 = block2.unravel_block()
        for cube in cubes_self:
            if cube in cubes_block2:
                return True
        return False
    
    def can_descend(self, block_list):
        lower_block1 = self.gen_lower_block()
        for block2 in block_list:
            if lower_block1.does_intersect(block2):
                return False
        
        return True
            
    def __eq__(self, block2):
            for idx in range(0,3):
                if self.edge1[idx] != block2.edge1[idx] or self.edge2[idx] != block2.edge2[idx]:
                    return False
            return True
                
    
    def __repr__(self):
        return str(self.edge1) + ' -> ' + str(self.edge2) + ' | n_blocks_above = ' + str(self.n_blocks_above_falling)


In [ ]:
def compact_tower(block_sorted):
    print('Compacting blocks...')
    block_compressed = []
    for idx, block1 in tqdm(enumerate(block_sorted)):
        # elements touching the ground are automatically added
        if block1.edge1[2] == 1:
            block_compressed.append(block1)
            continue
        # row 2 and higher
        while can_descend(block1, block_compressed[idx-1::-1]) and block1.edge1[2] > 1:
            block1 = block1.gen_lower_block()
        block_compressed.append(block1)
    print('Blocks compacted')
    return block_compressed

In [ ]:
def blocks_ending_at_layer(block_list, layer):
    blocks_ending_in_layer = []
    for block1 in block_list:
        # add all blocks ending at layer
        if block1.edge2[2] == layer:
            blocks_ending_in_layer.append(block1)
        # if we go above, we stop checking (to improve performance)
        if block1.edge1[2] > layer:
            break
    return blocks_ending_in_layer

In [ ]:
def blocks_beginning_at_layer(block_list, layer):
    blocks_beginning_in_layer = []
    for block1 in block_list:
        # add all blocks beginning at layer
        if block1.edge1[2] == layer:
            blocks_beginning_in_layer.append(block1)
        # if we go above, we stop checking (to improve performance)
        if block1.edge1[2] > layer:
            break
    return blocks_beginning_in_layer

In [ ]:
block_list = []

with open('Day22_input.txt') as f:
    for line in f:
        line = line.strip('\n').split('~')        
        block_list.append(Block(line[0], line[1]) )
        

block_sorted = sorted(block_list)

    
block_compact = compact_tower(block_sorted)

block_compact.sort()
for block in block_compact[:20]:
    print(block)

In [ ]:
n_removable_blocks = 0
n_cannot_be_removed = 0
# compute how many layers there are
n_layers = block_compact[-1].edge2[2]
n_blocks_checked = 0


for block in block_compact:
    block.n_blocks_above_falling = 0

# for each layer
# starting from the top to implement part 2 as well
for layer in tqdm(range(n_layers, 0, -1)):
    print('---------------------------------------- layer n ', layer,
         ' total blocks checked, including this top layer =', n_blocks_checked)
    # compute all the blocks ending in that layer
    blocks_ending_in_layer = blocks_ending_at_layer(block_compact, layer)
    # compute all the blocks starting in the next layer
    blocks_beginning_in_layer = blocks_beginning_at_layer(block_compact, layer+1)
    
    # if no blocks are ending in this layer
    if not blocks_ending_in_layer:
        continue
        
    print('\n\n* blocks_ending_in_layer: ', layer)
    for el in blocks_ending_in_layer:
        print(el)
    
    print('\n\n* blocks_beginning_in_layer: ', layer+1)
    for el in blocks_beginning_in_layer:
        print(el)
        
    print('\n\n')    
    
    # for each block in the layer
    for block1_bot in blocks_ending_in_layer:
        can_remove_block1_bot = True
        print('\n## checking block1_bot', block1_bot)
        # if there are no blocks starting at layer +1 --> should cover the top ones
        if not blocks_beginning_in_layer:
            print('++no blocks starting in layer ', layer +1, '  --> can remove', block1_bot)
            print('going to next block if any\n')
            n_removable_blocks +=1
            continue
        # generate list of bottom blocks without block1
        blocks_ending_in_layer_min_block1 = [temp for temp in blocks_ending_in_layer if temp != block1_bot ]        
        if not(blocks_ending_in_layer_min_block1):
            block1_bot.n_blocks_above_falling = n_blocks_checked -1
            print(block1_bot, ' was the only block ending at layer ', layer, ' cannot be removed')
            print('going to next layer\n')
            n_cannot_be_removed +=1
            continue
        
        # for all the blocks starting at the layer above
        for block_top in blocks_beginning_in_layer:
                print('checking who else can hold block_top ', block_top)
                if can_descend(block_top, blocks_ending_in_layer_min_block1):
                    block1_bot.n_blocks_above_falling += block_top.n_blocks_above_falling +1
                    print('-- ', block1_bot , ' is the only block holding ', block_top, ' and cannot be removed')
#                    print('moving to next block_top\n')
#                    n_cannot_be_removed +=1
                    can_remove_block1_bot = False
#                    break
                else:
                    print(block_top , ' can be held by another block')
        if can_remove_block1_bot:
            print('++ ',block1_bot, ' can be removed\n')
            n_removable_blocks +=1
    n_blocks_checked +=len(blocks_ending_in_layer)                   

sum_blocks_falling = 0
for block in tqdm(block_compact[::-1]):
    sum_blocks_falling += block.n_blocks_above_falling
        
print('total blocks checked = ', n_blocks_checked)
print('removable blocks = ',n_removable_blocks)
print('non removable blocks = ',n_cannot_be_removed)
print('total sum blocks falling = ', sum_blocks_falling)          
# more than 46055